## Install dependencies

In [3]:
import mediapipe as mp
import cv2 as cv


In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

## Get Webcam feed

In [22]:
# Connect to cam
cap = cv.VideoCapture(0)

with mp_holistic.Holistic( 
    min_detection_confidence=0.5, 
    min_tracking_confidence=0.5
) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Show image
        frame = cv.flip(frame, 1)
        image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)

        # Make detections
        results = holistic.process(image)
        #print(results.face_landmarks)

        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks

        # recolor image back to BGR for rendering
        image = cv.cvtColor(frame, cv.COLOR_RGB2BGR)

        # 1. Draw Face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS)

        # 2. Draw Right Hand landmarks
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                mp_drawing.DrawingSpec(color=(255,0,0), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(255,0,0), thickness=2, circle_radius=2))

        # 3. Draw Left Hand landmarks
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

        # 4. Draw Pose landmarks
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)




        cv.imshow('Webcam', image)

        # check if any keys are being pressed, if this is 'q' break out of while loop and release webcam & close frame
        if cv.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv.destroyAllWindows()

In [19]:
mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=2)

Init signature:
mp_drawing.DrawingSpec(
    color: Tuple[int, int, int] = (224, 224, 224),
    thickness: int = 2,
    circle_radius: int = 2,
) -> None
Docstring:      DrawingSpec(color: Tuple[int, int, int] = (224, 224, 224), thickness: int = 2, circle_radius: int = 2)
Source:        
@dataclasses.dataclass
class DrawingSpec:
  # Color for drawing the annotation. Default to the white color.
  color: Tuple[int, int, int] = WHITE_COLOR
  # Thickness for drawing the annotation. Default to 2 pixels.
  thickness: int = 2
  # Circle radius. Default to 2 pixels.
  circle_radius: int = 2
File:           c:\users\chris\anaconda3\envs\computer_vision\lib\site-packages\mediapipe\python\solutions\drawing_utils.py
Type:           type
Subclasses:     
